# Python with PostgreSQL & PostGIS

## Libraries and Settings

In [28]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

u:\Lektionen\GitHub_Repositories\python_postgresql_postgis


## Create database connection

In [29]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

('osm_switzerland',)


## List tables in database

In [30]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

planet_osm_point
geography_columns
geometry_columns
spatial_ref_sys
planet_osm_polygon
raster_columns
raster_overviews
planet_osm_line
municipalities_ch
planet_osm_roads
roads_zuerich_vertices_pgr
strassenlaerm_tag
roads_zuerich
route


## Show columns and data types of selected table

In [31]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

,column_name,data_type
0,osm_id,bigint
1,access,text
2,addr:housename,text
3,addr:street,text
4,addr:housenumber,text
...,...,...
68,width,text
69,wood,text
70,z_order,integer
71,way_area,real


## Query: Select buildings for which full address is available in defined zip code areas

In [32]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8001', '8002')"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


,osm_id,addr:street,addr:housenumber,addr:city,addr:postcode,building,geom
0,370058962,Waffenplatzstrasse,71c,Zürich,8002,roof,"POLYGON ((8.52737 47.35806, 8.52737 47.358, 8...."
1,149868409,Gablerstrasse,51,Zürich,8002,residential,"POLYGON ((8.52723 47.35836, 8.52727 47.35831, ..."
2,93184519,Waffenplatzstrasse,71,Zürich,8002,yes,"POLYGON ((8.52735 47.35822, 8.52741 47.35814, ..."
3,370058961,Waffenplatzstrasse,71b,Zürich,8002,yes,"POLYGON ((8.52753 47.35827, 8.52757 47.35823, ..."
4,370058960,Waffenplatzstrasse,71a,Zürich,8002,yes,"POLYGON ((8.52757 47.35805, 8.52759 47.35792, ..."
...,...,...,...,...,...,...,...
2351,32951943,Steinhaldenstrasse,68,Zürich,8002,apartments,"POLYGON ((8.52762 47.35759, 8.52762 47.35754, ..."
2352,370058964,Waffenplatzstrasse,73a,Zürich,8002,yes,"POLYGON ((8.52753 47.35772, 8.52753 47.35767, ..."
2353,370058963,Waffenplatzstrasse,71d,Zürich,8002,roof,"POLYGON ((8.5275 47.35791, 8.5275 47.35786, 8...."
2354,32951935,Waffenplatzstrasse,73,Zürich,8002,apartments,"POLYGON ((8.52712 47.35783, 8.52712 47.35772, ..."


## Show selected features on map

<span style="color: blue;">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span style="color: blue;">Example of alternative background maps (maptiles) are:</span>
- <span style="color: blue;">EsriWorldImagery</span>
- <span style="color: blue;">EsriWorldTopoMap</span>
- <span style="color: blue;">EsriWorldGrayCanvas</span>
- <span style="color: blue;">CartoDBDarkMatter</span>
- <span style="color: blue;">CartoDBPositron</span>


In [33]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='geojson',
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select coffee stores in Switzerland

In [34]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee';"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


,osm_id,shop,name,geom
0,10275946038,coffee,Quinta Coira,POINT (9.53125 46.8485)
1,4929337494,coffee,Badilatti,POINT (9.9654 46.59732)
2,4929338163,coffee,Rocca & Zgraggen Gastro-Maschinen,POINT (9.96413 46.59663)
3,2411403524,coffee,Goldcastle Tea & Coffee,POINT (6.95454 47.07048)
4,2473998249,coffee,Pappy John & Cie,POINT (6.65236 46.52688)
...,...,...,...,...
103,5434020963,coffee,Tchibo,POINT (7.24634 47.13622)
104,5434020954,coffee,Nespresso,POINT (7.24578 47.13613)
105,5611649752,coffee,Pelluch GmbH Kaffeemaschienen,POINT (7.54718 47.54892)
106,8909393062,coffee,Moccaraba,POINT (7.59199 47.56274)


## Show selected features on map

In [35]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=9, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span style="color: blue;">For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span style="color: blue;">In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span style="color: blue;">Note that WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) can be derived from:</span>

- <span style="color: blue;">https://tools.retorte.ch/map</span>


In [36]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


,osm_id,shop,name,distance_meters,geom
0,706203439,supermarket,Coop,159.883419,POINT (8.72594 47.50085)
1,4109460421,supermarket,Asia Shop,162.391281,POINT (8.72208 47.50101)
2,3831772784,supermarket,Migros,247.578208,POINT (8.72115 47.49916)
3,7380954145,supermarket,Alnatura,256.838011,POINT (8.72074 47.49958)
4,4095400190,supermarket,ALDI,274.275393,POINT (8.72476 47.4979)
5,4125136758,supermarket,Tandoor Indischer Supermarkt,290.212664,POINT (8.72017 47.50073)
6,4095400136,supermarket,Denner,316.354037,POINT (8.72036 47.49886)
7,709022324,supermarket,Claro Weltladen,441.129317,POINT (8.72912 47.49842)
8,4058248551,supermarket,Migros,600.117307,POINT (8.73193 47.50012)
9,3441033104,supermarket,L'Ultimo Bacio,680.202961,POINT (8.73299 47.49999)


## Show selected features on map

In [37]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all roads classified as 'primary' in Switzerland and create 5000m buffers around these roads

In [38]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
sql = """-- Create buffers around major roads and combine these buffers to one single buffer
        SELECT 
            1 as group_id,
            ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 5000)), 4326) AS geom
        FROM public.planet_osm_roads AS p
        WHERE
            --highway IN ('motorway', 'trunk', 'primary')
            highway IN ('motorway');"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

## Show selected features on map

In [39]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=9, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [40]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-10-01 18:46:29
Python Version: 3.11.9
-----------------------------------
